# 02 · Cleaning & Feature Engineering
Normalize JSON, build transactions and RFM features.

In [26]:
import pandas as pd, numpy as np, json, pathlib
DATA = pathlib.Path("../data")
products = pd.read_csv(DATA/"products.csv")
users = pd.read_csv(DATA/"users_raw.csv")
carts = pd.read_csv(DATA/"carts_raw.csv")
categories = pd.read_csv(DATA/"categories.csv")

# Normalize users (handles JSON-like strings)
def safe_to_json(x):
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", '"'))
        except Exception:
            return None
    return x if isinstance(x, dict) else None

users_name = users["name"].apply(safe_to_json)
users_addr = users["address"].apply(safe_to_json)

users_clean = pd.DataFrame({
    "user_id": users["id"],
    "email": users.get("email", pd.Series([None]*len(users))),
    "username": users.get("username", pd.Series([None]*len(users))),
    "firstname": users_name.apply(lambda d: d.get("firstname") if isinstance(d, dict) else None),
    "lastname": users_name.apply(lambda d: d.get("lastname") if isinstance(d, dict) else None),
    "city": users_addr.apply(lambda d: d.get("city") if isinstance(d, dict) else None),
    "zipcode": users_addr.apply(lambda d: d.get("zipcode") if isinstance(d, dict) else None),
})

# Explode carts -> transaction lines
carts["date"] = pd.to_datetime(carts["date"], errors="coerce")
lines = []
for _, row in carts.iterrows():
    prods = safe_to_json(row["products"])
    if isinstance(prods, list):
        for it in prods:
            lines.append({
                "cart_id": row["id"],
                "user_id": row["userId"],
                "date": row["date"],
                "product_id": it.get("productId"),
                "quantity": it.get("quantity", 1)
            })
tx = pd.DataFrame(lines)

# Join price & compute revenue
tx = tx.merge(products[["id","title","category","price"]], left_on="product_id", right_on="id", how="left")
tx["line_revenue"] = tx["quantity"] * tx["price"]
tx = tx.drop(columns=["id"]).sort_values("date")

# RFM - Recency, Frequency, Monetary
# Recency: days since last purchase
# Frequency: number of unique carts (orders)
# Monetary: total revenue
rfm = (tx.groupby("user_id")
         .agg(last_purchase=("date","max"),
              frequency=("cart_id","nunique"),
              monetary=("line_revenue","sum"))
         .reset_index())
rfm["recency_days"] = (tx["date"].max() - rfm["last_purchase"]).dt.days
rfm = rfm.drop(columns=["last_purchase"])

# Save curated
tx.to_csv(DATA/"transactions.csv", index=False)
users_clean.to_csv(DATA/"users.csv", index=False)
rfm.to_csv(DATA/"user_rfm.csv", index=False)
print("Saved transactions.csv, users.csv, user_rfm.csv in", DATA)

## Summary statistics
## Normalize, data structure, missing values, outliers, duplicates, distributions 
## EDA: distributions, correlations, trends, patterns, anomalies
## Feature engineering: RFM, customer segments, product categories, time-based features
## Data quality: missing values, duplicates, inconsistencies, outliers
## Documentation: data dictionary, code comments, README

Saved transactions.csv, users.csv, user_rfm.csv in ../data


#### Clean The 'Users' Table ####

In [ ]:
# Check data types, missing values, basic stats
users_clean.info()
print(users_clean.isnull().sum())
print(users_clean.describe(include='all'))  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    10 non-null     int64 
 1   email      10 non-null     object
 2   username   10 non-null     object
 3   firstname  10 non-null     object
 4   lastname   10 non-null     object
 5   city       10 non-null     object
 6   zipcode    10 non-null     object
dtypes: int64(1), object(6)
memory usage: 692.0+ bytes
user_id      0
email        0
username     0
firstname    0
lastname     0
city         0
zipcode      0
dtype: int64
         user_id           email username firstname lastname      city  \
count   10.00000              10       10        10       10        10   
unique       NaN              10       10         9       10         9   
top          NaN  john@gmail.com    johnd     david      doe  kilcoole   
freq         NaN               1        1         2        1         2   
mean 

In [ ]:
users_clean['city'].value_counts().head(10)

In [ ]:
users_clean['city'] = users_clean['city'].str.title().str.strip()
users_clean['city'].value_counts().head(10)

city
Kilcoole       2
San Antonio    2
Cullman        1
El Paso        1
Fresno         1
Mesa           1
Miami          1
Fort Wayne     1
Name: count, dtype: int64

#### Clean the 'Transactions' table

In [22]:
tx.info()
print(tx.isna().sum())  
print(tx.select_dtypes(include=[np.number]).describe())

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 8 to 2
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   cart_id       14 non-null     int64              
 1   user_id       14 non-null     int64              
 2   date          14 non-null     datetime64[ns, UTC]
 3   product_id    14 non-null     int64              
 4   quantity      14 non-null     int64              
 5   title         14 non-null     object             
 6   category      14 non-null     object             
 7   price         14 non-null     float64            
 8   line_revenue  14 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(4), object(2)
memory usage: 1.1+ KB
cart_id         0
user_id         0
date            0
product_id      0
quantity        0
title           0
category        0
price           0
line_revenue    0
dtype: int64
         cart_id    user_id  product_i

#### Clean the 'Products' table

In [33]:
# Check products table data types, missing values, basic stats
products.info()
print(products.isna().sum())
print(products.select_dtypes(include=[np.number]).describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20 non-null     int64  
 1   title         20 non-null     object 
 2   price         20 non-null     float64
 3   description   20 non-null     object 
 4   category      20 non-null     object 
 5   image         20 non-null     object 
 6   rating_rate   20 non-null     float64
 7   rating_count  20 non-null     int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 1.4+ KB
id              0
title           0
price           0
description     0
category        0
image           0
rating_rate     0
rating_count    0
dtype: int64
             id       price  rating_rate  rating_count
count  20.00000   20.000000    20.000000     20.000000
mean   10.50000  162.046000     3.560000    286.800000
std     5.91608  272.220532     0.961578    162.751538
min     1.00000    7.950000  

In [ ]:
# Explode rating column into two separate columns (rate, count)
products["rating"] = products["rating"].apply(safe_to_json)
products["rating_rate"] = products["rating"].apply(lambda d: d.get("rate") if isinstance(d, dict) else None)
products["rating_count"] = products["rating"].apply(lambda d: d.get("count") if isinstance(d, dict) else None)
products.drop(columns=["rating"], inplace=True)
products.head() 

,id,title,price,description,category,image,rating_rate,rating_count
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


In [28]:
products.select_dtypes(include='object').describe()

,title,description,category,image,rating
count,20,20,20,20,20
unique,20,20,4,20,20
top,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",Your perfect pack for everyday use and walks i...,electronics,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
freq,1,1,6,1,1
